Workbook for final capstone of IBM Data Science Certification

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import numpy as np
from geopy import geocoders
from geopy import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    altair-4.0.0               |             py_0         606 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be 

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
df_towns = pd.read_csv(body)
df_towns['Lat'] = '1.1' #create a latitude column
df_towns['Long'] = '1.1' #create a longitude column
df_towns.iloc[13,0] = 'Santa Rosa'#sonoma to santa rosa due to Sonoma returning the middle of nowhere
df_towns.iloc[17,0] = 'Hilton Head Island' #hilton head doesn't register for Foursquare, renaming to recognized name
df_towns.iloc[1,0] = 'Jackson' #Jackson hole returns a location in the middle of Teton National forest, renaming to the correct town
df_towns.iloc[2,0] = 'South Lake Tahoe' #Lake Tahoe gives you the middle of literal lake Tahoe. Renaming to the actual town
geolocator = Nominatim(user_agent='myapplication')
for l in range(len(df_towns.index)): #populate latitude and longitude values for each town
    town = df_towns.iloc[l,0] + ' ' + df_towns.iloc[l,1]
    location = geolocator.geocode(town)
    df_towns.iloc[l,2] = location.latitude
    df_towns.iloc[l,3] = location.longitude

In [4]:
df_towns.iloc[24,2] = 38.935631 ##reassigning locations based on real data
df_towns.iloc[24,3] = -74.913718 #Cape May is way off to the north
df_towns.iloc[6,2] = 36.600262 ##reassigning locations based on real data
df_towns.iloc[6,3] = -121.879785 ##Monterey is way off to the south

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000): ##function like in New york to run through the towns
    #radius has to be extended due to not being a city
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            200)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Town', 
                  'Town Latitude', 
                  'Town Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
town_venues = getNearbyVenues(names=df_towns['Town'], latitudes=df_towns['Lat'],longitudes=df_towns['Long']) ##runs the above function for each town

In [7]:
towns_onehot = pd.get_dummies(town_venues[['Venue Category']], prefix="", prefix_sep="")# one hot encoding
towns_onehot['Town'] = town_venues['Town'] # add town column back to dataframe
fixed_columns = [towns_onehot.columns[-1]] + list(towns_onehot.columns[:-1])
towns_onehot = towns_onehot[fixed_columns]
towns_grouped = towns_onehot.groupby('Town').mean().reset_index()
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_categories = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Town']
for ind in np.arange(num_top_categories):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
towns_venues_sorted = pd.DataFrame(columns=columns)
towns_venues_sorted['Town'] = towns_grouped['Town']

for ind in np.arange(towns_grouped.shape[0]):
    towns_venues_sorted.iloc[ind, 1:] = return_most_common_venues(towns_grouped.iloc[ind, :], num_top_categories)

In [8]:
from collections import Counter
column_list = ['Town', '1st Most Common Word', '2nd Most Common Word', '3rd Most Common Word', '4th Most Common Word', '5th Most Common Word','6th Most Common Word', '7th Most Common Word', '8th Most Common Word', '9th Most Common Word', '10th Most Common Word']
word_count = pd.get_dummies(column_list, prefix="", prefix_sep="")
word_list = pd.DataFrame(columns=['Town', 'Word'])
word_count = word_count[['Town', '1st Most Common Word', '2nd Most Common Word', '3rd Most Common Word', '4th Most Common Word', '5th Most Common Word','6th Most Common Word', '7th Most Common Word', '8th Most Common Word', '9th Most Common Word', '10th Most Common Word']]
filler_words = ['&', 'the', 'at', 'of', 'and', '-', 'telluride', 'jackson', 'tahoe', 'sedona', 'bar', 'harbor', 'lake', 'gatlinburg', 'monterey', 'steamboat', 'springs', 'williamsburg', 'moab', 'stowe', 'juneau', 'laguna', 'santa', 'rosa', 'breckenridge', 'sanibel', 'island', 'martha\'s', 'vineyard', 'hilton', 'head', 'island', 'st.', 'augustine', 'destin', 'myrtle', 'ocean', 'city', 'beach', 'cannon', 'charlottesville', 'cape', 'may', 'hole', '0', '1',
               '2', '3', '4', '5', '6', '7', '8', '9', '10']#list of words with no real meaning that I don't want to consider. Also includes the names of locations
for l in range(14): ##adds rows to the dataframe for all towns
    rowname = 11 + l
    word_count = word_count.append(pd.Series(name= rowname))

for l in range(len(df_towns.index)): ##generates two dataframes:  a nice clean one for viewing, and a larger one for onehot encoding
    word_count.iloc[l,0] = df_towns.iloc[l,0]
    certain_words = town_venues.loc[town_venues['Town']==word_count.iloc[l,0]]
    certain_word_counts = pd.DataFrame(Counter(" ".join(certain_words['Venue'].str.lower()).split()).most_common(50))
    certain_word_counts_clean = certain_word_counts[~certain_word_counts[0].isin(filler_words)]
    for j in range(10):
        word_count.iloc[l,j+1] = certain_word_counts_clean.iloc[j,0]
    for k in range(len(certain_word_counts.index)):
        count = certain_word_counts.iloc[k,1]
        for m in range(count):
            word_list = word_list.append({'Town' : df_towns.iloc[l,0], 'Word': certain_word_counts.iloc[k,0]}, ignore_index=True)

word_list_clean = word_list[~word_list['Word'].isin(filler_words)] ##removes filler words

In [9]:
#onehot for word_list_clean
words_onehot = pd.get_dummies(word_list_clean[['Word']], prefix="", prefix_sep="")# one hot encoding
words_onehot['Town'] = word_list_clean['Town'] # add town column back to dataframe
fixed_columns = [words_onehot.columns[-1]] + list(words_onehot.columns[:-1])
words_onehot = words_onehot[fixed_columns]
words_grouped = words_onehot.groupby('Town').mean().reset_index()

We now have two data sets, the most common words and the most common venues categories. I will run clustering on three data sets: words, categories, and the two combined. I will test with various amount of clusters and determine the best answer using them all

Three data sets: df_combined has both words and locations, word_counts has only words, and town_venues_sorted has location categories

## Below: Merged Data Set

In [10]:
df_combined = towns_venues_sorted.join(word_count.set_index('Town'), on='Town') #combines the two data sets, have to do before processing sub-components
df_combined_loc = df_combined.join(df_towns[['Town','Lat', 'Long']].set_index('Town'), on='Town', how = 'right') #above data set plus location information for presentation table
one_hot_combined = towns_grouped.join(words_grouped.set_index('Town'), how = 'right')

In [11]:
# set number of clusters
kclusters = 8

merged_clustered = one_hot_combined.drop('Town', 1)
merged_clustered.fillna(0, inplace = True)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged_clustered)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
df_combined_loc.insert(0, 'Merged Cluster Labels', kmeans.labels_)
df_combined_loc.head(25) # check the last columns!

,Merged Cluster Labels,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,3rd Most Common Word,4th Most Common Word,5th Most Common Word,6th Most Common Word,7th Most Common Word,8th Most Common Word,9th Most Common Word,10th Most Common Word,Lat,Long
0,3,Bar Harbor,Seafood Restaurant,Trail,Hotel,American Restaurant,Ice Cream Shop,Mountain,Park,Beach,...,cafe,hotel,lobster,trail,street,inn,cove,desert,44.3865,-68.2119
1,0,Breckenridge,Ski Area,Coffee Shop,Hotel,Ski Chairlift,American Restaurant,Bar,Ski Lodge,Beer Garden,...,center,superchair,restaurant,blue,ski,summit,mountain,coffee,39.4817,-106.038
2,1,Cannon Beach,Beach,Hotel,American Restaurant,Coffee Shop,Seafood Restaurant,Resort,State / Provincial Park,Brewery,...,inn,restaurant,state,arch,coffee,company,lounge,ecola,45.8918,-123.962
3,5,Cape May,American Restaurant,Beach,Seafood Restaurant,Hotel,Coffee Shop,Ice Cream Shop,Campground,Breakfast Spot,...,company,congress,house,sunset,wildwood,inn,washington,hall,38.9356,-74.9137
4,5,Charlottesville,Brewery,American Restaurant,Music Venue,Coffee Shop,Burger Joint,Pizza Place,Mexican Restaurant,Bagel Shop,...,kitchen,park,bodo's,bagels,coffee,monticello,center,beer,38.0293,-78.4767
5,5,Destin,Seafood Restaurant,Beach,Resort,Grocery Store,Pizza Place,Coffee Shop,Mexican Restaurant,Bar,...,house,park,henderson,mexican,grill,resort,home,starbucks,30.3935,-86.4958
6,4,Gatlinburg,Resort,Hotel,Scenic Lookout,Breakfast Spot,Candy Store,Trail,BBQ Joint,Ski Chairlift,...,falls,mountain,ole,center,bear,great,pancake,cabin,35.7143,-83.5102
7,4,Hilton Head Island,Beach,Resort,American Restaurant,Pizza Place,Hotel,Restaurant,Seafood Restaurant,Park,...,sea,palmetto,dunes,cove,harbour,pines,shelter,park,32.1618,-80.7513
8,6,Jackson,Hotel,American Restaurant,Coffee Shop,Grocery Store,Resort,Sandwich Place,Bar,BBQ Joint,...,teton,pearl,street,hotel,center,grill,pizza,bakery,43.4743,-110.766
9,5,Juneau,Seafood Restaurant,Coffee Shop,Bar,Pizza Place,American Restaurant,Café,Dive Bar,Mexican Restaurant,...,company,cafe,brewing,co.,heritage,salmon,treadwell,adventure,58.3019,-134.42


In [12]:
address = 'United States of America' #create a user agent

geolocator = Nominatim(user_agent="tourist_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

merged_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
# set color scheme for the clusters
xm = np.arange(kclusters)
ysm = [i + xm + (i*xm)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ysm)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_combined_loc['Lat'], df_combined_loc['Long'], df_combined_loc['Town'], df_combined_loc['Merged Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(merged_map_clusters)
       
merged_map_clusters

## Below: venue categories

In [13]:
# set number of clusters
kclusters = 8

category_clustered = towns_grouped.drop('Town', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(category_clustered)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
towns_venues_sorted.insert(0, 'Venue Cluster Labels', kmeans.labels_)
towns_venues_sorted_loc = towns_venues_sorted.join(df_towns[['Town','Lat', 'Long']].set_index('Town'), on='Town', how = 'right')
towns_venues_sorted_loc.head(25) # check the last columns!

,Venue Cluster Labels,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Lat,Long
0,6,Bar Harbor,Seafood Restaurant,Trail,Hotel,American Restaurant,Ice Cream Shop,Mountain,Park,Beach,Breakfast Spot,Brewery,44.3865,-68.2119
1,5,Breckenridge,Ski Area,Coffee Shop,Hotel,Ski Chairlift,American Restaurant,Bar,Ski Lodge,Beer Garden,Restaurant,Pizza Place,39.4817,-106.038
2,7,Cannon Beach,Beach,Hotel,American Restaurant,Coffee Shop,Seafood Restaurant,Resort,State / Provincial Park,Brewery,Restaurant,Trail,45.8918,-123.962
3,0,Cape May,American Restaurant,Beach,Seafood Restaurant,Hotel,Coffee Shop,Ice Cream Shop,Campground,Breakfast Spot,Italian Restaurant,Deli / Bodega,38.9356,-74.9137
4,2,Charlottesville,Brewery,American Restaurant,Music Venue,Coffee Shop,Burger Joint,Pizza Place,Mexican Restaurant,Bagel Shop,Park,Hotel,38.0293,-78.4767
5,3,Destin,Seafood Restaurant,Beach,Resort,Grocery Store,Pizza Place,Coffee Shop,Mexican Restaurant,Bar,Furniture / Home Store,Bookstore,30.3935,-86.4958
6,1,Gatlinburg,Resort,Hotel,Scenic Lookout,Breakfast Spot,Candy Store,Trail,BBQ Joint,Ski Chairlift,American Restaurant,Donut Shop,35.7143,-83.5102
7,7,Hilton Head Island,Beach,Resort,American Restaurant,Pizza Place,Hotel,Restaurant,Seafood Restaurant,Park,Harbor / Marina,Grocery Store,32.1618,-80.7513
8,2,Jackson,Hotel,American Restaurant,Coffee Shop,Grocery Store,Resort,Sandwich Place,Bar,BBQ Joint,Breakfast Spot,Pizza Place,43.4743,-110.766
9,6,Juneau,Seafood Restaurant,Coffee Shop,Bar,Pizza Place,American Restaurant,Café,Dive Bar,Mexican Restaurant,Mountain,Trail,58.3019,-134.42


In [14]:
category_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(towns_venues_sorted_loc['Lat'], towns_venues_sorted_loc['Long'], towns_venues_sorted_loc['Town'], towns_venues_sorted_loc['Venue Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(category_map_clusters)
       
category_map_clusters

Venue categories only

## Below: Most common words

In [15]:
# set number of clusters
kclusters = 8

words_clustered = words_grouped.drop('Town', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(words_clustered)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
word_count.insert(0, 'Word Cluster Labels', kmeans.labels_)
word_count_loc = word_count.join(df_towns[['Town','Lat', 'Long']].set_index('Town'), on='Town', how = 'right')
word_count_loc.head(25) # check the last columns!

,Word Cluster Labels,Town,1st Most Common Word,2nd Most Common Word,3rd Most Common Word,4th Most Common Word,5th Most Common Word,6th Most Common Word,7th Most Common Word,8th Most Common Word,9th Most Common Word,10th Most Common Word,Lat,Long
0,3,Telluride,gondola,station,mountain,sheridan,lodge,village,brown,pizza,cafe,new,37.9375,-107.812
1,0,Jackson,restaurant,inn,teton,pearl,street,hotel,center,grill,pizza,bakery,43.4743,-110.766
2,1,South Lake Tahoe,cafe,heavenly,coffee,grill,dog,pizza,edgewood,hut,south,lodge,38.9332,-119.984
3,5,Sedona,rock,resort,trail,creek,spa,park,oak,red,cafe,trailhead,34.8697,-111.761
4,5,Bar Harbor,park,acadia,cafe,hotel,lobster,trail,street,inn,cove,desert,44.3865,-68.2119
5,5,Gatlinburg,smoky,resort,falls,mountain,ole,center,bear,great,pancake,cabin,35.7143,-83.5102
6,4,Monterey,bay,del,monte,grill,park,market,house,state,co.,old,36.6003,-121.88
7,4,Steamboat Springs,restaurant,grill,resort,creek,express,chairlift,brewery,cafe,mountain,storm,40.4848,-106.832
8,6,Williamsburg,shop,tavern,house,grill,market,square,cafe,dog,palace,colonial,37.2789,-76.6945
9,5,Moab,park,center,inn,suites,arches,trail,cafe,arch,campground,national,38.5738,-109.546


In [16]:
word_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)
# set color scheme for the clusters
xw = np.arange(kclusters)
ysw = [i + xw + (i*xw)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ysw)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(word_count_loc['Lat'], word_count_loc['Long'], word_count_loc['Town'], word_count_loc['Word Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(word_map_clusters)
       
word_map_clusters